# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities = pd.read_csv("baig_cities.csv", encoding="utf-8")
cities.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Bengkulu,-3.80,102.27,75.42,80.76,84,1011,4.27,63
1,Eyl,7.98,49.82,76.32,75.34,66,1013,10.42,63
2,Ponta do Sol,32.67,-17.10,59.02,53.26,59,1014,8.05,63
3,Zavetnoye,47.12,43.89,46.42,34.70,69,1014,15.82,63
4,Port Elizabeth,-33.92,25.57,57.20,54.97,87,1011,5.82,63


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
humidity = cities["Humidity"].astype(float)
locations = cities[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
0,-3.80,102.27
1,7.98,49.82
2,32.67,-17.10
3,47.12,43.89
4,-33.92,25.57
...,...,...
540,64.50,-165.41
541,39.45,-86.18
542,24.09,32.91
543,29.16,16.14


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
cities.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Bengkulu,-3.80,102.27,75.42,80.76,84,1011,4.27,63
1,Eyl,7.98,49.82,76.32,75.34,66,1013,10.42,63
2,Ponta do Sol,32.67,-17.10,59.02,53.26,59,1014,8.05,63
3,Zavetnoye,47.12,43.89,46.42,34.70,69,1014,15.82,63
4,Port Elizabeth,-33.92,25.57,57.20,54.97,87,1011,5.82,63


In [10]:
cities.describe()

,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
count,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.000000,545.0
mean,19.491321,16.879927,58.633945,55.332606,72.225688,1012.946789,8.289156,63.0
std,33.408915,92.049823,22.358855,27.473492,19.869854,8.344981,6.074582,0.0
min,-54.800000,-175.200000,-15.050000,-22.650000,10.000000,946.000000,0.400000,63.0
25%,-8.140000,-63.060000,46.400000,37.940000,63.000000,1009.000000,3.830000,63.0
50%,22.290000,18.960000,63.860000,59.700000,77.000000,1013.000000,6.930000,63.0
75%,47.470000,100.480000,77.000000,78.960000,87.000000,1017.000000,11.410000,63.0
max,78.220000,179.320000,94.600000,96.890000,100.000000,1034.000000,44.960000,63.0


In [12]:
mask = (cities.Temperature < 70) & (cities.Temperature > 66) & (cities["Wind Speed"] < 5)
df_sub = cities.loc[mask].reset_index(drop=True)
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Bilma,18.69,12.92,66.67,60.91,25,1016,3.31,63
1,Bambous Virieux,-20.34,57.76,68.02,71.13,88,1020,3.36,63
2,Souillac,-20.52,57.52,66.20,68.58,88,1020,3.36,63
3,Kaeo,-35.10,173.78,68.00,72.00,93,1011,3.00,63
4,Mahébourg,-20.41,57.70,66.20,68.58,88,1020,3.36,63


In [13]:
df_sub.City.values

array(['Bilma', 'Bambous Virieux', 'Souillac', 'Kaeo', 'Mahébourg',
       'Saint-Philippe', 'Hamilton', 'Beloha', 'Akonolinga',
       'Saint-Joseph', 'Araxá', 'Ngunguru', 'Fontem'], dtype=object)

In [15]:
coords = df_sub[["Latitude", "Longitude"]]
coords_ideal = df_sub[["Latitude", "Longitude"]]
humids = df_sub.Humidity

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=df_sub.City)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
#inits
names = []
addresses = []
# prices = []
# ratings = []

# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in df_sub.iterrows():
    
    location = f"{row.Latitude}, {row.Longitude}"

    # add keyword to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row.City}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
              
        name = results[0]['name']
        address = results[0]['vicinity']
#         price = results[0]['price_level']
#         rating = results[0]['rating']
              
        names.append(name)
        addresses.append(address)
#         prices.append(price)
#         ratings.append(rating)
              
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append(np.nan)
        addresses.append(np.nan)
#         prices.append(np.nan)
#         ratings.append(np.nan)
        
    print("------------")
              
df_sub["hotel_name"] = names
df_sub["address"] = addresses
# df_sub["price_level"] = prices
# df_sub["rating"] = ratings

Retrieving Results for Index 0: Bilma.
Closest Hotel is Централна джамия Билма.
------------
Retrieving Results for Index 1: Bambous Virieux.
Closest Hotel is Football Ground.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Souillac.
Closest Hotel is Souillac Health Office.
------------
Retrieving Results for Index 3: Kaeo.
Closest Hotel is Spice Grill Indian Cuisine.
------------
Retrieving Results for Index 4: Mahébourg.
Closest Hotel is Hamilton College Boys Ville Noire.
------------
Retrieving Results for Index 5: Saint-Philippe.
Closest Hotel is Le Palmier.
------------
Retrieving Results for Index 6: Hamilton.
Closest Hotel is John "Johnny Appleseed" Chapman Monument.
------------
Retrieving Results for Index 7: Beloha.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Akonolinga.
Closest Hotel is Health Center Sainte Salomé.
------------
Retrieving Results for Index 9: Saint-Joseph.
Closest Hotel is The Bird House.
----

In [17]:
hotelList = []

for i in range(len(df_sub)):
    lat = df_sub.loc[i]['Latitude']
    lng = df_sub.loc[i]['Longitude']

    params = {
        "rankby": "distance",
        "type" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotelList.append(jsn['results'][0]['name'])
    except:
        hotelList.append("")
#df_sub["Hotel Name"] = hotelList
#df_sub = df_sub.dropna(how='any')
df_sub.head()

,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness,hotel_name,address
0,Bilma,18.69,12.92,66.67,60.91,25,1016,3.31,63,Централна джамия Билма,Bilma
1,Bambous Virieux,-20.34,57.76,68.02,71.13,88,1020,3.36,63,NaN,NaN
2,Souillac,-20.52,57.52,66.20,68.58,88,1020,3.36,63,Souillac Health Office,"Savanne Road, Souillac"
3,Kaeo,-35.10,173.78,68.00,72.00,93,1011,3.00,63,Spice Grill Indian Cuisine,"20, Leigh Street, Kaeo"
4,Mahébourg,-20.41,57.70,66.20,68.58,88,1020,3.36,63,Hamilton College Boys Ville Noire,"Pres de l'Aeroport, Mahebourg"


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in df_sub.iterrows()]
locations = df_sub[["Latitude", "Longitude"]]

In [27]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…